In [ ]:

###############################################################################
# Random Forest Classification Model (TensorFlow)                             #
# For LOIC Dataset                                                            #
# Based on the Implementation of:                                             #
# https://www.tensorflow.org/decision_forests/tutorials/beginner_colab        #
###############################################################################


In [1]:
# Installieren der benötigten Python Pakete
!pip install numpy==1.19.2
!pip install six==1.15.0
!pip install wheel==0.35
!pip install pandas
!pip install tensorflow_decision_forests
!pip install wurlitzer
!pip install matplotlib
!pip install dask[complete]

     |████████████████████████████████| 722 kB 2.6 MB/s 
     |████████████████████████████████| 10.7 MB 12.7 MB/s 
     |████████████████████████████████| 97 kB 9.0 MB/s 
  Using cached tornado-6.1-cp38-cp38-manylinux2010_x86_64.whl (427 kB)
     |████████████████████████████████| 296 kB 36.9 MB/s 
     |████████████████████████████████| 302 kB 28.2 MB/s 
  Using cached Jinja2-3.0.1-py3-none-any.whl (133 kB)
  Using cached MarkupSafe-2.0.1-cp38-cp38-manylinux2010_x86_64.whl (30 kB)
  Created wheel for bokeh: filename=bokeh-2.3.2-py3-none-any.whl size=11334267 sha256=0ee20a9f65328734dbdd43b26aa9780889a6277d8bbbce32563a00d6c45d7405
  Stored in directory: /home/julianbuecher/.cache/pip/wheels/1b/a5/90/f38b6cd80a8276a7203765295ad3da078aa24ff8006096ae49
Successfully built bokeh


In [2]:
# Importieren der benötigten Python Pakete
import pandas as pd
import numpy as np
import tensorflow_decision_forests as tfdf
from wurlitzer import sys_pipes
import matplotlib.pyplot as plt
import dask.dataframe as dd

ModuleNotFoundError: No module named 'dask'

In [ ]:
# Laden der Netzwerk Traffic Daten für den LOIC Angriff
data_LOIC = pd.read_csv('../Data/Optimized/Tuesday-20-02-2018_LOIC-Attack.csv')

In [ ]:
# Festlegen der Label-Spalte innerhalb des Datasets
label = 'label'

In [ ]:
# Aufteilen des Datasets in Training- und Test-Daten
def split_dataset(dataset,  test_ratio=0.30):
    """Splits a panda dataframe in two dataframes."""
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]

training_data_LOIC, testing_data_LOIC = split_dataset(data_LOIC)

print("{} examples in training, {} examples for testing.".format(
    len(training_data_LOIC), len(testing_data_LOIC)))

In [ ]:
# Konvertieren des Panda Dataframes in ein TensorFlow Dataset
print("Converting Panda Dataframe into TensorFlow Dataset...")
training_dataset_LOIC = tfdf.keras.pd_dataframe_to_tf_dataset(training_data_LOIC, label=label)
testing_dataset_LOIC = tfdf.keras.pd_dataframe_to_tf_dataset(testing_data_LOIC, label=label)

In [ ]:
# Erstellen des Random Forest Modells
model = tfdf.keras.RandomForestModel()
model.compile(metrics=["accuracy"])

In [ ]:
# Trainieren des Modells
print("Training the Model...")
with sys_pipes():
    model.fit(x=training_dataset_LOIC)

In [ ]:
# Evaluieren des trainierten Modells
print("Evaluating the Model...")
evaluation = model.evaluate(testing_dataset_LOIC, return_dict=True)
print()
for name, value in evaluation.items():
    print(f"{name}: {value:.4f}")


In [ ]:
# Erstellen einer Bilanz für das trainierte Modell
model.summary()

In [ ]:
# Erstellen der Feature Importance Kriterien aus Sicht des TensorFlow Modells
model.make_inspector().variable_importances()